In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
table = pd.read_html(link)

In [3]:
table[0].head()

,0,1,2,3,4,5,6,7,8
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park / Ontario Provincial Government,M8ANot assigned,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),M7BNot assigned,M8BNot assigned,M9BEtobicoke(West Deane Park / Princess Garden...
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M2CNot assigned,M3CNorth York(Don Mills)South(Flemingdon Park),M4CEast York(Woodbine Heights),M5CDowntown Toronto(St. James Town),M6CYork(Humewood-Cedarvale),M7CNot assigned,M8CNot assigned,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,M1EScarborough(Guildwood / Morningside / West ...,M2ENot assigned,M3ENot assigned,M4EEast Toronto(The Beaches),M5EDowntown Toronto(Berczy Park),M6EYork(Caledonia-Fairbanks),M7ENot assigned,M8ENot assigned,M9ENot assigned
4,M1GScarborough(Woburn),M2GNot assigned,M3GNot assigned,M4GEast York(Leaside),M5GDowntown Toronto(Central Bay Street),M6GDowntown Toronto(Christie),M7GNot assigned,M8GNot assigned,M9GNot assigned


### 'For' loop to rewrite every 'not assigned' into a np.nan

In [4]:
for column in table[0].columns:
    for i in range(table[0].shape[0]):
        if 'assigned' in table[0][column].iloc[i]:
            table[0][column].iloc[i] = np.nan

In [5]:
table[0].head()

,0,1,2,3,4,5,6,7,8
0,NaN,NaN,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park / Ontario Provincial Government,NaN,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),NaN,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),NaN,NaN,M9BEtobicoke(West Deane Park / Princess Garden...
2,M1CScarborough(Rouge Hill / Port Union / Highl...,NaN,M3CNorth York(Don Mills)South(Flemingdon Park),M4CEast York(Woodbine Heights),M5CDowntown Toronto(St. James Town),M6CYork(Humewood-Cedarvale),NaN,NaN,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,M1EScarborough(Guildwood / Morningside / West ...,NaN,NaN,M4EEast Toronto(The Beaches),M5EDowntown Toronto(Berczy Park),M6EYork(Caledonia-Fairbanks),NaN,NaN,NaN
4,M1GScarborough(Woburn),NaN,NaN,M4GEast York(Leaside),M5GDowntown Toronto(Central Bay Street),M6GDowntown Toronto(Christie),NaN,NaN,NaN


### 'For' loop to extract from the table the values of postal code, borough and neighborhood

In [6]:
postal_code = []
borough = []
neighborhood = []
for column in table[0].columns:
    postal_code.append(table[0][column].apply(lambda x: str(x)[0:3]))
    borough.append(table[0][column].apply(lambda x: str(x)[3:].split('(')[0]))
    for neigh in (table[0][column].apply(lambda x: str(x)[3:].split('('))):
                  if len(neigh) > 1:
                       neighborhood.append(neigh[1][:-1])
                  else:
                        neighborhood.append(neigh[0])

In [7]:
#'For' loop to get rid of NaN values in the list of lists postal_code
PostalCode = []
for lista in postal_code:
    for code in lista:
        if code != 'nan':
            PostalCode.append(code)

In [8]:
#'For' loop to get rid of empty values (len(value) == 0) in the list of lists borough
Borough = []
for lista in borough:
    for b in lista:
        if len(b) > 0:
            Borough.append(b)

In [9]:
#'For' loop to get rid of empty values (len(value) == 0) in the list neighborhood
Neighborhood = []
for i in neighborhood:
    if len(i) > 0:
        Neighborhood.append(i)

In [10]:
d = {'PostalCode':PostalCode, 'Borough':Borough, 'Neighborhood':Neighborhood}
df = pd.DataFrame(d)
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Retrieving the latitude and longitude coordinates of each neighborhood

In [11]:
lat_lng_df = pd.read_csv('http://cocl.us/Geospatial_data')
lat_lng_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
#As state below, the postal code column of the latitude and longitude dataframe is the same as the df's postal code column,
#so we can just add the columns of interest into the df
sum(lat_lng_df['Postal Code'] == df['PostalCode'])

103

In [13]:
df['Latitude'] = lat_lng_df['Latitude']
df['Longitude'] = lat_lng_df['Longitude']

In [16]:
#The final Data Frame
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [15]:
df.shape

(103, 5)